In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from ydata_profiling import ProfileReport
import seaborn as sns
import unicodedata as unicodedata
sns.set()

C:\Users\Mapas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv('df_arreglado.csv')

In [5]:
df.head()

,Unnamed: 0,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,...,id_collection,name_collection,id_genres,name_genres,id_production,name_production,id_countrie,name_countrie,id_language,name_language
0,0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,...,10194.0,Toy Story Collection,"16.0,35.0,10751.0","Animation,Comedy,Family",3.0,Pixar Animation Studios,US,United States of America,en,English
1,1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,...,NaN,NaN,"12.0,14.0,10751.0","Adventure,Fantasy,Family","559.0,2550.0,10201.0","TriStar Pictures,Teitler Film,Interscope Commu...",US,United States of America,"en,fr","English,Français"
2,2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,...,119050.0,Grumpy Old Men Collection,"10749.0,35.0","Romance,Comedy","6194.0,19464.0","Warner Bros.,Lancaster Gate",US,United States of America,en,English
3,3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,...,NaN,NaN,"35.0,18.0,10749.0","Comedy,Drama,Romance",306.0,Twentieth Century Fox Film Corporation,US,United States of America,en,English
4,4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,...,96871.0,Father of the Bride Collection,35.0,Comedy,"5842.0,9195.0","Sandollar Productions,Touchstone Pictures",US,United States of America,en,English


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45376 entries, 0 to 45375
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         45376 non-null  int64  
 1   budget             45376 non-null  float64
 2   id                 45376 non-null  int64  
 3   original_language  45365 non-null  object 
 4   overview           44435 non-null  object 
 5   popularity         45376 non-null  float64
 6   release_date       45376 non-null  object 
 7   revenue            45376 non-null  float64
 8   runtime            45130 non-null  float64
 9   status             45296 non-null  object 
 10  tagline            20398 non-null  object 
 11  title              45376 non-null  object 
 12  vote_average       45376 non-null  float64
 13  release_year       45376 non-null  int64  
 14  return             45376 non-null  float64
 15  id_collection      4488 non-null   float64
 16  name_collection    448

# API 1 

In [70]:
def peliculas_mes(mes):
    fechas = pd.to_datetime(df['release_date'], format= '%Y-%m-%d')
    n_mes = fechas[fechas.dt.strftime('%B').str.capitalize() == mes.capitalize()]
    respuesta = n_mes.shape[0]
    return {'mes':mes, 'cantidad':respuesta}


In [71]:
peliculas_mes('marzo')

{'mes': 'marzo', 'cantidad': 3553}

# API 2

In [72]:
def peliculas_dia(dia):
    fechas = pd.to_datetime(df['release_date'], format= '%Y-%m-%d')
    n_dia= fechas[fechas.dt.day_name(locale = 'es')==dia.capitalize()]
    respuesta = n_dia.shape[0]
    return {'dia':dia, 'cantidad':respuesta}

In [73]:
peliculas_dia('lunes')

{'dia': 'lunes', 'cantidad': 3503}

# API 3

In [74]:
def franquicia(franquicia):
    f_bajo= franquicia.lower()
    fran= df[['name_collection','budget','revenue',]].dropna(subset=['name_collection'])
    fran= fran[fran['name_collection'].map(str.lower).apply(lambda x: f_bajo in x)]
    cantidad = fran.shape[0]
    gananciat= (fran['revenue']- fran['budget']).sum()
    gananciap= (fran['revenue']- fran['budget']).mean()
    return {'franquicia': franquicia, 'cantidad': cantidad, 'ganancia_total':gananciat, 'ganancia_promedio': gananciap}
               

In [75]:
franquicia('Toy Story Collection')

{'franquicia': 'Toy Story Collection',
 'cantidad': 3,
 'ganancia_total': 1617890605.0,
 'ganancia_promedio': 539296868.3333334}

# API 4

In [19]:
def peliculas_pais(pais):
    m1 = df[['title', 'name_countrie']]
    if isinstance(pais,str):
        pais = pais.lower()
        pais = unicodedata.normalize('NFKD', pais).encode('ascii', 'ignore').decode('utf-8','ignore')
        ganancias = m1['title'][m1['name_countrie'].str.contains(pais)==True]
        respuesta = ganancias.shape[0]
    return {'pais': pais, 'cantidad': respuesta} # convine este

In [21]:
def peliculas_pais(pais: str) -> dict:
    if isinstance(pais, str):
        pais = pais.lower()
        pais = unicodedata.normalize('NFKD', pais).encode('ascii', 'ignore').decode('utf-8','ignore')
        cantidad = df['name_countrie'].apply(lambda x: str(x).lower()).map(str.lower).apply(lambda x: pais in x).sum()
        return {'pais': pais, 'cantidad': cantidad} # con este

In [23]:
def peliculas_pais(pais):
    m1 = df[['title', 'name_countrie']]
    if isinstance(pais, str):
        pais = pais.lower()
        pais = unicodedata.normalize('NFKD', pais).encode('ascii', 'ignore').decode('utf-8','ignore')
        cantidad = m1['title'][m1['name_countrie'].str.contains(pais)==True]
        cantidad = df['name_countrie'].apply(lambda x: str(x).lower()).map(str.lower).apply(lambda x: pais in x)
        respuesta = cantidad.shape[0]
    return {'pais': pais, 'cantidad': respuesta} # y salio al fin este 

In [24]:
peliculas_pais ('United States of America')

{'pais': 'united states of america', 'cantidad': 45376}

# API 5

In [78]:
def productoras(productora):
    prod = df[['name_production', 'budget', 'revenue']].dropna()
    prod ['name_production'] = prod['name_production'].map(str.lower)
    cantidad = prod.shape[0]
    gtotal= (prod['revenue'] - prod['budget']).sum()
    return {'productora':productora, 'ganancia_total': gtotal, 'cantidad': cantidad }




In [79]:
productoras('Warner Bros.')

{'productora': 'Warner Bros.',
 'ganancia_total': 317931314253.0,
 'cantidad': 33580}

# API 6

In [80]:
def retorno(pelicula):
    peli = df[['title','budget', 'revenue', 'return', 'release_year']].dropna()
    peli['title'] = peli['title'].map(str.lower)
    peli = peli[peli['title']==str(pelicula).lower()]
    inver = peli['budget'].loc[0]
    gan = peli['revenue'].loc[0]
    ret = peli['return'].loc[0]
    an = peli['release_year'].loc[0]
    return {'pelicula': pelicula, 'inversion': inver, 'ganancia': gan, 'retorno': ret, 'anio': an } # este no anda

In [81]:
def retorno(pelicula: str) -> dict:
    pelicula_df = df.loc[df['title'] == pelicula.title()]
    inversion = pelicula_df['budget'].iloc[0].item()
    ganancia = pelicula_df['revenue'].iloc[0].item()
    retorno= pelicula_df['return'].iloc[0].item()
    anio = pelicula_df['release_year'].iloc[0].item()
    return {'pelicula': pelicula, 'inversion': inversion, 'ganancia': ganancia, 'retorno': retorno, 'anio': anio } # arreglado



In [82]:
retorno('toy story')

{'pelicula': 'toy story',
 'inversion': 30000000.0,
 'ganancia': 373554033.0,
 'retorno': 12.4518011,
 'anio': 1995}

# API 7